Firstly I import all libraries that will be needed during the project.

In [20]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import re
print('Libraries imported.')

Libraries imported.


Now I read Wiki page into <i>wiki1</i>, then find the needed table using regular expression and finally handle the header and all rows using regular expressions. As result I get the table from Wikipedia page in a form of DataFrame <i>df</i>.

In [21]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
re1 = "<table[^>]*><tbody><tr><th>Postcode.*M9Z</td><td>Not assigned</td><td>Not assigned</td></tr></tbody></table>"
wiki1 = requests.get(url).text.replace('\n','')
table = re.search(re1, wiki1).group(0)
columns = re.findall("<th>([^<>]*)</th>",table)
rows = re.findall("(<tr><td>(?:(?!<tr>).)*</tr>)", table)
df = pd.DataFrame(columns=(columns+['Link']))
for row in rows:
    re2 = '<td>([^<>]*)</td><td>(?:<a[^<>]*>)*([^<>]*)(?:</a>)*</td><td>(?:<a[^<>]*href\s*=\s*"([^"]*)"[^>]*>)*([^<>]*)(?:</a>)*</td>'
    values = re.findall(re2, row)
    norm_values = [tuple((values[0][0],values[0][1],values[0][3],values[0][2]))]
    df = pd.concat([df,pd.DataFrame(norm_values,columns=df.columns.values)],axis=0)
df.reset_index(drop=True,inplace=True)

Next I convert the table in accordance to the task.

In [23]:
df2 = df[df['Borough']!='Not assigned'].drop('Link',axis=1)
df2.columns = ['PostalCode','Borough','Neighborhood']
df2['Neighborhood'].replace(to_replace='Not assigned', value=df2['Borough'], inplace=True)
PostalCodes = df2.groupby('PostalCode').agg({'Borough': 'first',
                                             'Neighborhood': lambda x: ', '.join(x)})
PostalCodes.reset_index(inplace=True)
PostalCodes.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [24]:
PostalCodes.shape

(103, 3)